<div class="alert alert-success" role="alert">
  <h1 class="alert-heading">Análisis Vibracional</h1>
  <h3 class="alert-heading">Prof. Enrique Mejía Ospino, emejia@uis.edu.co</h3>
  <h4 class="alert-heading">Escuela de Química</h4>
  <h4 class="alert-heading">Universidad Industrial de Santander</h4>
  </div>

**<font color=blue> Vamos  atilizar el modulo *PSI4* para desarrollar algunos ejercicios de cálculo mecánico-cuántico. En este Cuaderno de Jupyter realizaremos análisis vibracional de algunas moleculas sencillas.** 

In [1]:
# Importamos modulos =================================================================
import psi4
import numpy as np  
import matplotlib.pyplot as plt
import h5py
import fortecubeview # Visualizar moléculas
import pandas as pd
%matplotlib inline

In [2]:
# Inicialización Psi4 =================================================================

psi4.core.clean()
psi4.core.clean_options()
psi4.set_memory('2000 MB')  # Can make this much larger on Seawulf, each compute node has more than 100 GB RAM
psi4.set_num_threads(4)    # Can make this much larger on Seawulf, each compute node can support 24 threads. 
                          # But it doesn't help much for small molecules...
psi4.core.set_output_file('./Data/OCS_pythonics.dat', False) 
# this command sets psi4's output to a file. Comment this line out if you want to see the output on the terminal.

In [3]:
# Geometría de la molécula de OCS ============================================
OCS_120 = psi4.geometry("""
 o
 c    1 co2     
 s    2 sc3         1 sco3      
 
co2=        1.380000
sc3=        1.780000
sco3=       120.000
""")
OCS_120.reset_point_group('c1') # simetría!

In [8]:
# Inicialización de los cálculos ================================================

methods = ['scf/3-21G', 'scf/6-311G*'] # Lista de métodos. El último será usado para la otimización!
mol_names = ['OCS_linear' , 'OCS_bent'] # Simetrías

# Initialización de diccionario de Python para almacenar los resultados ======================
    
molecules = {} # Para almacenar las moléculas
E0 = {}        # Para almacenar la energías no optimizadas
Eopt = {}      # Para almacenar las energías optimizadas
wfn_opt = {}   # Para almacenar la funciones de onda optimizadas

# Geometría de la molécula de OCS ============================================
molecules['OCS_bent'] = psi4.geometry("""
 o
 c    1 co2     
 s    2 sc3         1 sco3      
 
co2=        1.380000
sc3=        1.780000
sco3=       120.000
""")

molecules['OCS_bent'].reset_point_group('c1') # 
E0['OCS_bent'] = {} # Inicializamos
Eopt['OCS_bent'] = 0 #

# ejemplo de cargar desde un archivo
f = open('./Data/OCS_linear.xyz') #usamos la función ´open´ de Python para cargar un archivo xyz.
molecules['OCS_linear'] = psi4.geometry(f.read())
molecules['OCS_linear'].reset_point_group('c1') # 
E0['OCS_linear'] = {} # Inicializamos
Eopt['OCS_linear'] = 0 # 

# Corremos los calculos. Usamos un loop ========================

for mol_name in mol_names:
    print('Working on ' + mol_name)    
    for meth_j in methods:                 #loop over methods
        E0[mol_name][meth_j] = psi4.energy(meth_j, molecule = molecules[mol_name])
    
    # usamos esta parate para controlar casos especiales
    if mol_name == 'OCS_bent':
        psi4.set_module_options('optking',{'frozen_bend': '1 2 3'})
        
    Eopt[mol_name], wfn_opt[mol_name] = psi4.optimize(methods[-1], molecule = molecules[mol_name], 
                                                      return_wfn = True)
    psi4.driver.molden(wfn_opt[mol_name], './Data/' + mol_name + '.molden')


# Impresión de Resultados en pantalla =================================================================

print('Energías en posiciones iniciales con diferentes métodos:')
print(E0)
print(methods[-1] + ' energies after optimization:')
print(Eopt)

#%%
# Guardar los resultados optimizados usando Pandas
df = pd.DataFrame(data = Eopt, index = [0]) # DataFrame
df.to_excel('./Data/Lab1starter_Eopt.xlsx')  # guardamos en un archivo excel.

#%%
# Guardamos los resultados iniciales usando Pandas
df = pd.DataFrame(E0)
df.to_excel('./Data/Lab1starter_E0.xlsx')  #

Working on OCS_linear
Optimizer: Optimization complete!
Working on OCS_bent
Optimizer: Optimization complete!
Energías en posiciones iniciales con diferentes métodos:
{'OCS_bent': {'scf/3-21G': -507.5063893205801, 'scf/6-311G*': -510.1193187715536}, 'OCS_linear': {'scf/3-21G': -507.57113391033636, 'scf/6-311G*': -510.18337850323314}}
scf/6-311G* energies after optimization:
{'OCS_bent': -510.21076698642946, 'OCS_linear': -510.3125229088429}


In [ ]:
# Save data to a spreadsheet using Pandas ==========================================
#
#  d = {'Details': [method1, method2, 'Global opt.'], 'Energies': [E0_1, E0_2, Eopt]} # create dictionary to load into Pandas dataframe.
#  df = pd.DataFrame(data = d)     # create Pandas dataframe
#  df.to_csv('Lab1starter_OCS_results.csv')  # save dataframe to .csv spreadsheet file.
 
# Or of course you can also extract the results from the raw .dat file
# generated by Psi4